<a href="https://colab.research.google.com/github/cooolbabu/GoogleGemini101/blob/main/OpenAI/SimpleChatWithPromptTemplate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Chat Models


In [ ]:
%pip install langchain langchain_openai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.5/238.5 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 req

In [ ]:
# import os
# os.environ['OPENAI_API_KEY'] = 'API_KEY_HERE'

# Used to securely store your API key
from google.colab import userdata

OPENAI_API_KEY=userdata.get('OpenAI_KEY')

In [ ]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

# If you have an envionrment variable set for OPENAI_API_KEY, you can just do:
# chat = ChatOpenAI()

# Let's have a simple chat

In [ ]:
chat.invoke("Hello, how are you?")

AIMessage(content="Hello! I'm an AI, so I don't have feelings, but I'm here to help you. How can I assist you today?")

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]
result = chat.invoke(messages)

In [ ]:
print(type(result))
print(result)
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
content='Rainbow Threads'
Rainbow Threads


---

## Inserting the SystemMessage instead of a HumanMessage


In [ ]:
text = "What would be a good company name for a company that makes colorful socks?"

messages = [SystemMessage(content=text)]
chatResponse = chat(messages)
print(chatResponse)

content='VibrantSock'


In [ ]:
print(type(chatResponse))
print(chatResponse)
print(chatResponse.content)

<class 'langchain_core.messages.ai.AIMessage'>
content='VibrantSock'
VibrantSock


# Let have a chatbot that uses prompt template.
- We want brand names for our shoes. Different countries have their localized brand name
- Pass country name as a parameter and generate local brand
- Specify that output format is a JSON with root element as brand_names

In [ ]:

# Country name is a parameter.

prompt_template = "You are shoe marketing specialist in {country}. What would be catch brand name for shoes. Add country native flavor. Suggest three names and return result in json format with name and reason. Use brand_names as root element"


In [ ]:
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
)
import pandas as pd

system_message_prompt = SystemMessagePromptTemplate.from_template(prompt_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])


In [ ]:
# Generate suggestions for Italian market. 'brand_name' is specified as the root element
italian_suggestions = chat.invoke(
    chat_prompt.format_prompt(country="Italy").to_messages()
)

df = pd.DataFrame(json.loads(italian_suggestions.content)['brand_names'])
df

,name,reason
0,Soleil,This name combines the word 'sole' (Italian fo...
1,Passeggiata,Passeggiata means 'stroll' or 'walk' in Italia...
2,Dolce Passione,Dolce Passione translates to 'sweet passion' i...


In [ ]:
#
# Generate suggestions for Mexican market
mexican_suggestions = chat.invoke(
    chat_prompt.format_prompt(country="Mexico").to_messages()
)

df = pd.DataFrame(json.loads(mexican_suggestions.content)['brand_names'])
df

,name,reason
0,Zapatos Azteca,This brand name combines the Spanish word for ...
1,Piedra Maya,Piedra Maya means 'Mayan Stone' in Spanish. Th...
2,Sole Mexicano,This brand name combines the English word 'Sol...


## Parsers
- In the previous example we specified the output format to be a JSON object ith brand_names as the root element
- In this section let's see how we can use Langchain parsers
- There are a variety of Output parsers
  - https://python.langchain.com/docs/modules/model_io/output_parsers/
  - Here let's take a look at Pydantic, JSON and Pandas dataframe

- Very useful when calling functions and data needs to be in a structured format


# What is the difference between OpenAI and ChatOpenAI

- OpenAI class includes more generic machine learning task attributes such as frequency_penalty, presence_penalty, logit_bias, allowed_special, disallowed_special, best_of.

- ChatOpenAI class provides more chat-related methods, such as completion_with_retry, get_num_tokens_from_messages to make it more user-friendly when build chatbot related applications.

[Good article in Stack Overflow](https://stackoverflow.com/questions/76950609/what-is-the-difference-between-openai-and-chatopenai-in-langchain)

